# Water Project : MySQL Table Creation
The weather data has been gathered from WeatherAPI.com. For each station, we have stored the weather as a DataFrame in a .pickle file.
  
We need to store the weather data in a MySQL database on GCP :
- WeatherAPI history can't be accessed without a subscription
- requests a SQL is easier and quicker

A MySQL database is hosted on GCP

The following task are done in this Notebook :
- MySQL connection setup, table created
- DataFrame Loaded from '.pickle' files and reshaped
- The MySQL table is populated

In [10]:
from dotenv import dotenv_values
import os, re

import sqlalchemy
from sqlalchemy import create_engine, text
import pandas as pd

## MySQL Setup and Table Creation
- We use a mysql database on Google Cloud Platform
- sqlalchemy engine is setup from variables stored in .env file
- We create the table

In [11]:
# MYSQL CONFIGURATION

settings = dotenv_values() # Loads settings from .env file
ROOT='..' # relative path to the root of the project

db_uri = (
    f"mysql+pymysql://{settings['SQL_USER']}:{settings['SQL_PWD']}"
    f"@{settings['SQL_HOST']}/{settings['SQL_DB']}"
    f"?ssl_ca={os.path.join(ROOT,settings['SQL_SSL_CA'])}"
    f"&ssl_cert={os.path.join(ROOT,settings['SQL_SSL_CERT'])}"
    f"&ssl_key={os.path.join(ROOT,settings['SQL_SSL_KEY'])}"
    f"&ssl_check_hostname=false"
)

engine = create_engine(db_uri,echo=False, future=False)

In [13]:
# TABLE CREATION

create_query = \
"""
create TABLE weather (
    day DATE NOT NULL,
    station_id INT NOT NULL,
    temperature DECIMAL(3,1) NOT NULL,
    precipitation DECIMAL(5,1) NOT NULL,
    maxwind DECIMAL(4,1) NOT NULL,
    description VARCHAR(255) NOT NULL,
    CONSTRAINT pk PRIMARY KEY (day,station_id)
);
"""

with engine.connect() as conn :
    result = conn.execute(text(create_query))

## Loading Weather History from .pickle files
- WeaterAPI has been used to constitute the history of weather for all the stations
- for each station, the data is stored in a .pickle file as a DataFrame
- file name is formatted as '<10_digits_station_id>.pickle'
- each DataFrame needs index / columns shaping to match the DB table

In [14]:
# List of the files in '../stations' folder 

STATIONS_DIR = os.path.join('..','data','stations')
els = os.listdir(STATIONS_DIR) # elements in the dir
files = [ e for e in els if os.path.isfile(os.path.join(STATIONS_DIR,e)) ]

# Restrains the list to well formatted names
pattern = re.compile(r'^[0-9]{7}(?=.pickle)')
files = [ f for f in files if pattern.match(f) ]

# Lists the ids and paths
ids = [ pattern.match(f).group() for f in files ]
paths = [ os.path.join(STATIONS_DIR,f) for f in files ]

In [17]:
# Reads the files and store the DataFrame in a dict
stations = { st_id:pd.read_pickle(path) for st_id,path in zip(ids,paths) }

# converts index, column names to match the mysql table
# and adds the station ID column
cols = ['day','temperature','precipitation','maxwind','description']

for station_id,df in stations.items():
    
    # inplace modifications mandatory
    # (df in for loop is just a assigned variable)
    df.reset_index(inplace=True) 
    df.columns = cols
    df['station_id'] = station_id

## Populating the Table
We use pandas with the sqlalchemy engine to populate the table with the data

In [18]:
for df in stations.values() :
    df.to_sql('weather',engine,if_exists='append',index=False)

Below are two examples on how to load data from mysql to pandas :
- entire table
- from a query

In [19]:
# Full sql table

weather = pd.read_sql_table('weather',con=engine)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70080 entries, 0 to 70079
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   day            70080 non-null  datetime64[ns]
 1   station_id     70080 non-null  int64         
 2   temperature    70080 non-null  float64       
 3   precipitation  70080 non-null  float64       
 4   maxwind        70080 non-null  float64       
 5   description    70080 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 3.2+ MB


In [20]:
# From a query

query = """
SELECT * FROM weather
WHERE
    day BETWEEN '2012-01-01' AND '2012-12-31'
    AND station_id = 6017070
"""
weather_sample = pd.read_sql(query,engine)
weather_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            366 non-null    object 
 1   station_id     366 non-null    int64  
 2   temperature    366 non-null    float64
 3   precipitation  366 non-null    float64
 4   maxwind        366 non-null    float64
 5   description    366 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 17.3+ KB
